In [ ]:
# Clone GitHub repo
!git clone https://github.com/vsyrgkanis/adversarial_reisz.git
%cd adversarial_reisz
# !git checkout charitable-giving

Cloning into 'adversarial_reisz'...
remote: Enumerating objects: 479, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 479 (delta 87), reused 121 (delta 60), pack-reused 313
Receiving objects: 100% (479/479), 9.95 MiB | 25.78 MiB/s, done.
Resolving deltas: 100% (248/248), done.
/content/adversarial_reisz
Branch 'charitable-giving' set up to track remote branch 'charitable-giving' from 'origin'.
Switched to a new branch 'charitable-giving'


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install econml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 51.1 MB/s eta 0:00:00


In [ ]:
!python setup.py develop

running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.



In [ ]:
import os
import joblib
from google.colab import files

import numpy as np
import pandas as pd

from pathlib import Path
import matplotlib.pyplot as plt

import scipy
import scipy.special
from sklearn.linear_model import LassoCV, LogisticRegressionCV, LinearRegression, Lasso, LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone
import torch
import torch.nn as nn
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils.multiclass import type_of_target

import statsmodels.api as sm  # For probit
from utilities import mean_ci
import scipy.stats as sps
import statistics

from debiased import DebiasedMoment
from advreisz.linear import SparseLinearAdvRiesz
from advreisz.kernel import AdvNystromKernelReisz, AdvKernelReisz, NystromKernelReisz, KernelReisz
from advreisz.deepreisz import AdvReisz
from advreisz.ensemble import AdvEnsembleReisz, AdvEnsembleReiszRegVariant, RFrr, interactive_poly_feature_fns
from utilities import AutoKernel, prod_kernel, PluginRR, PluginRR2, FitParamsWrapper

from experiments import *

GPU: True


In [ ]:
%matplotlib inline

# Regression Estimator

In [ ]:
def get_reg_fn(X, y):
    est = Pipeline([('p', PolynomialFeatures(degree=2, include_bias=False)),
                    ('s', StandardScaler()),
                    ('lasso', LassoCV(max_iter=10000, random_state=123))])
    est.fit(X, y)

    return lambda: Pipeline([('p', PolynomialFeatures(degree=2, include_bias=False)),
                             ('s', StandardScaler()),
                             ('lasso', Lasso(alpha=est.named_steps['lasso'].alpha_, max_iter=10000, random_state=123))])

# Riesz Estimators (Not Dependent On #Treatments)
*The plugin logistic, plugin RF, and adversarial RF estimators depend on the
number of "treatments", the variables that are counterfactually varied. All other Riesz estimators do not. See the moment function with two treatment for more details.*

In [ ]:
def get_splin_fn(X):
    return lambda: SparseLinearAdvRiesz(moment_fn,
                                        featurizer=Pipeline([('p', PolynomialFeatures(degree=2, include_bias=False)),
                                                             ('s', StandardScaler()),
                                                             ('cnt', PolynomialFeatures(degree=1, include_bias=True))]),
                                        n_iter=50000, lambda_theta=0.01, B=10,
                                        tol=0.00001)

def get_advnyskernel_fn(X):
    n_components = 100
    est = AdvNystromKernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]),
                                regm='auto', regl='auto', n_components=n_components, random_state=123)
    reg = est.opt_reg(X)
    return lambda: AdvNystromKernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]),
                                         regm=6*reg, regl=reg, n_components=n_components, random_state=123)

def get_advnyskernel_fn_1000(X):
    n_components = 1000
    est = AdvNystromKernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]),
                                regm='auto', regl='auto', n_components=n_components, random_state=123)
    reg = est.opt_reg(X)
    return lambda: AdvNystromKernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]),
                                         regm=6*reg, regl=reg, n_components=n_components, random_state=123)

# def get_advkernel_fn(X):  # Very computationally intensive; more practical to use a Nystrom approximation with 1000 components instead (see above)
#     est = AdvKernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]), regm='auto', regl='auto')
#     reg = est.opt_reg(X)
#     return lambda: AdvKernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]), regm=6*reg, regl=reg)

device = torch.cuda.current_device() if torch.cuda.is_available() else None
print("GPU:", torch.cuda.is_available())

# Returns a deep model for the reisz representer
def get_learner(n_t, n_hidden, p):
    return nn.Sequential(nn.Dropout(p=p), nn.Linear(n_t, n_hidden), nn.LeakyReLU(),
                         nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.LeakyReLU(),
                         nn.Dropout(p=p), nn.Linear(n_hidden, 1))

# Returns a deep model for the test functions
def get_adversary(n_z, n_hidden, p):
    return nn.Sequential(nn.Dropout(p=p), nn.Linear(n_z, n_hidden), nn.ReLU(),
                         nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.ReLU(),
                         nn.Dropout(p=p), nn.Linear(n_hidden, 1))

def get_agmm_fn(X):
    torch.manual_seed(123)
    n_hidden = 100
    dropout = 0.5
    return lambda: FitParamsWrapper(AdvReisz(get_learner(X.shape[1], n_hidden, dropout),
                                             get_adversary(X.shape[1], n_hidden, dropout),
                                             moment_fn),
                                   val_fr=.2,
                                   preprocess_epochs=200,
                                   earlystop_rounds=100,
                                   store_test_every=20,
                                   learner_lr=1e-4, adversary_lr=1e-4,
                                   learner_l2=6e-4, adversary_l2=1e-4,
                                   n_epochs=1000, bs=100,
                                   logger=None, model_dir=str(Path.home()), device=device, verbose=1)

GPU: True


# Charitable Giving Data Results
*This section conducts analysis on the charitable giving data of "Does Price Matter in Charitable Giving? Evidence from a Large-Scale Natural Field Experiment" (Karlan & List, 2007).*

### Moment Function (Two Treatments) – Diff-in-Diff
*Both variables D and A are treatments here, and the moment function of interest is a classic difference-in-differences function.*

In [ ]:
# E[E[Y|D=1, A=1, X] – E[Y|D=0, A=1, X] – (E[Y|D=1, A=0, X] – E[Y|D=0, A=0, X])]
# D is the first column, A is the second, and X is the remaining columns.
def moment_fn(x, test_fn):
    n_obs = x.shape[0]
    if torch.is_tensor(x):
        with torch.no_grad():
            t11 = torch.cat([torch.ones((n_obs, 2)).to(device), x[:, 2:]], dim=1)
            t01 = torch.cat([torch.zeros((n_obs, 1)).to(device), torch.ones((n_obs, 1)).to(device), x[:, 2:]], dim=1)
            t10 = torch.cat([torch.ones((n_obs, 1)).to(device), torch.zeros((n_obs, 1)).to(device), x[:, 2:]], dim=1)
            t00 = torch.cat([torch.zeros((n_obs, 2)).to(device), x[:, 2:]], dim=1)
    else:
        t11 = np.hstack([np.ones((n_obs, 2)), x[:, 2:]])
        t01 = np.hstack([np.zeros((n_obs, 1)), np.ones((n_obs, 1)), x[:, 2:]])
        t10 = np.hstack([np.ones((n_obs, 1)), np.zeros((n_obs, 1)), x[:, 2:]])
        t00 = np.hstack([np.zeros((n_obs, 2)), x[:, 2:]])
    return test_fn(t11) - test_fn(t01) - test_fn(t10) + test_fn(t00)

### Riesz Estimators (Two Treatments)
*The Riesz estimators for two treatments are defined here.*

In [ ]:
def get_lg_plugin_fn(X):
    clf = LogisticRegressionCV(cv=3, max_iter=10000, random_state=123)
    C_ = clf.fit(X[:, 2:], X[:, 1]).C_[0]
    model_t_A = LogisticRegression(C=C_, max_iter=10000, random_state=123)
    clf = LogisticRegressionCV(cv=3, max_iter=10000, random_state=123)
    C_ = clf.fit(X[:, 1:], X[:, 0]).C_[0]
    model_t_treat = LogisticRegression(C=C_, max_iter=10000, random_state=123)
    return lambda: PluginRR2(model_t_A=model_t_A, model_t_treat=model_t_treat,
                             min_propensity=1e-6)

def get_rf_plugin_fn(X):
    gcv = GridSearchCV(RandomForestClassifier(bootstrap=True, random_state=123),
                       param_grid={'max_depth': [3, None],
                                   'min_samples_leaf': [10, 50]},
                       scoring='r2',
                       cv=5)
    best_model_A = clone(gcv.fit(X[:, 2:], X[:, 1]).best_estimator_)
    best_model_treat = clone(clone(gcv).fit(X[:, 1:], X[:, 0]).best_estimator_)
    return lambda: PluginRR2(model_t_A=best_model_A,
                             model_t_treat=best_model_treat,
                             min_propensity=1e-6)

def get_rf_fn(X):
    return lambda: AdvEnsembleReisz(moment_fn=moment_fn,
                                    n_treatments=2,
                                    max_abs_value=15,  # originally 26
                                    n_iter=40,
                                    degree=1)

### Load Data

In [ ]:
# get data
np.random.seed(123)
df = pd.read_stata('https://github.com/gsbDBI/ExperimentData/raw/master/Charitable/RawData/AER%20merged.dta',
                   convert_categoricals=False)
df = df.loc[(df['ratio'] == 0) | (df['ratio'] == 1)]
df = df.drop(['control', 'ratio', 'ratio2', 'ratio3',
            'size', 'size25', 'size50', 'size100', 'sizeno',
            'ask', 'askd1', 'askd2', 'askd3', 'ask1', 'ask2', 'ask3',
            'amountchange', 'state50one', 'blue0'], axis=1)
# state50one just tags one (arbitrary?) observation for each state
# blue0 and red0 and perfectly collinear (when all variables are nonmissing); bluecty and redcty are not
df = df.dropna()
y_amount = df['amount'].values
y_gave = df['gave'].values
X = df[['treatment', 'red0',
        'hpa', 'year5', 'dormant', 'nonlit', 'cases', 'redcty', 'bluecty',
        'pwhite', 'pblack', 'page18_39', 'ave_hh_sz', 'median_hhincome', 'powner', 'psch_atlstba', 'pop_propurban']].values

In [ ]:
# synthetic
synthetic = False

if synthetic:

    def true_fn(X):
        return X[:, 0] + X[:, 0] * X[:, 1] + X[:, 1] + .1 * X[:, 2]

    true = np.mean(moment_fn(X, true_fn))
    scale = np.std(true_fn(X)) / 2
    print(true, scale)
    y_amount = true_fn(X) + np.random.normal(0, scale, size=(X.shape[0],))

### Scaling, Shuffling, and (for Two Treatments) Propensity-Score Filtering

In [ ]:
np.random.seed(123)

# scale non-binary variables
y_amount = y_amount.astype(np.double)
y_gave = y_gave.astype(np.double)
X = X.astype(np.double)
idx_nonbi = [i for i in range(2, X.shape[1]) if type_of_target(X[:, i]) != 'binary']  # indices of non-binary variables (first and second columns should be binary)
X[:, idx_nonbi] = StandardScaler().fit_transform(X[:, idx_nonbi])
y_scale = np.std(y_amount)
y_amount = y_amount / y_scale

# shuffle data to get random partitions for cross-validation (doing it once at the start is more efficient than repeatedly doing it; the cross-validation here and elsewhere assumes that the data is shuffled)
inds = np.arange(X.shape[0])
np.random.shuffle(inds)
X, y_amount, y_gave = X[inds].copy(), y_amount[inds].copy(), y_gave[inds].copy()

# drop extreme party and "treatment" (offered-match) propensities
clf_party = LogisticRegressionCV(cv=5, max_iter=10000, random_state=123).fit(X[:, 2:], X[:, 1])
clf_treat = LogisticRegressionCV(cv=5, max_iter=10000, random_state=123).fit(X[:, 1:], X[:, 0])
prop_party = clf_party.predict_proba(X[:, 2:])[:, 1]
prop_treat = clf_treat.predict_proba(X[:, 1:])[:, 1]
filt = (prop_party <= .9) & (prop_party >= .1) & (prop_treat <= .9) & (prop_treat >= .1)
print(X.shape[0], np.sum(filt))
X, y_amount, y_gave = X[filt], y_amount[filt], y_gave[filt]

25859 21712


### Estimation (AER)
*The analysis here uses the same regressors as Table 6, Column 9 of Karlan and List (2007). The purpose is, for the purposes of comparison, to get analogous results for the "two treatment" case where both offered-match and party-of-state are counterfactually varied.*

In [ ]:
# This is to replicate set of the regressors using in Table 6, Column 9 in the AER paper
def process_regressors(X_old):
    """Interacts the controls and then adds a constant"""
    return sm.add_constant(np.append(X_old, X_old[:, [0]] * X_old[:, 1:], axis = 1), has_constant = 'add')

def moment_fn_withprocessing(x, test_fn):
    n_obs = x.shape[0]
    t11 = process_regressors(np.hstack([np.ones((n_obs, 2)), x[:, 2:]]))
    t01 = process_regressors(np.hstack([np.zeros((n_obs, 1)), np.ones((n_obs, 1)), x[:, 2:]]))
    t10 = process_regressors(np.hstack([np.ones((n_obs, 1)), np.zeros((n_obs, 1)), x[:, 2:]]))
    t00 = process_regressors(np.hstack([np.zeros((n_obs, 2)), x[:, 2:]]))
    return test_fn(t11) - test_fn(t01) - test_fn(t10) + test_fn(t00)

In [ ]:
alpha = 0.05
n_resamples = 1000

# Get data
vars = ['treatment', 'red0', 'pwhite', 'pblack',
        'page18_39', 'ave_hh_sz', 'median_hhincome',
        'powner', 'psch_atlstba', 'pop_propurban']
X_AER = df[vars][filt]
X_AER = X_AER.values

# Calculate critical value
critval = sps.norm(loc=0, scale=1).ppf(1-alpha/2)

# OLS
ols_model = sm.OLS(y_amount*y_scale, process_regressors(X_AER)).fit() # unscale y
p = ols_model.params[11]
s = ols_model.bse[11]
l = p - critval * s
u = p + critval * s
ols_results = {'point': p, 'stderr': s,
               'lower': l, 'upper': u}

# Probit (with bootstrapped standard errors)
def probit_results_fn(y, X):
  probit_model = sm.Probit(y, process_regressors(X)).fit()
  moment_pred = moment_fn_withprocessing(X, probit_model.predict)
  return mean_ci(moment_pred, confidence = 1-alpha)

p, s, l, u = probit_results_fn(y_gave, X_AER)

p_boot_list = []
for i in range(n_resamples):
  idx = np.random.choice(np.arange(len(y_gave)), len(y_gave), replace = True)
  X_boot = X_AER[idx, :]
  y_boot = y_gave[idx]
  p_boot, s_boot, l_boot, u_boot = probit_results_fn(y_boot, X_boot)
  p_boot_list.append(p_boot)

s = statistics.stdev(p_boot_list)
l = p - critval * s
u = p + critval * s

probit_results = {'point': p, 'stderr': s,
                  'lower': l, 'upper': u}

In [ ]:
print(ols_results)
print(probit_results)

### Estimation (Riesz)

In [ ]:
def do_analysis_charitable(y, n_splits):
    res = {}
    for name, get_reisz_fn in [
                                ('plugin_lg', get_lg_plugin_fn),
                                ('plugin_rf', get_rf_plugin_fn),
                                ('splin', get_splin_fn),
                                # ('advrkhs', get_advkernel_fn),
                                ('nys_advrkhs_1000', get_advnyskernel_fn_1000),
                                ('nys_advrkhs', get_advnyskernel_fn),
                                ('advrf', get_rf_fn),
                                ('advnnet', get_agmm_fn),
                                ]:
        est = DebiasedMoment(moment_fn=moment_fn,
                                get_reisz_fn=get_reisz_fn,
                                get_reg_fn=get_reg_fn, n_splits=n_splits)
        est.fit(X, y)
        p, s, l, u = est.avg_moment()
        res[name] = {'point': p * y_scale, 'stderr': s * y_scale,
                                'lower': l * y_scale, 'upper': u * y_scale}

    res = pd.DataFrame(res).transpose()
    res

In [ ]:
do_analysis_charitable(y = y_amount, n_splits = 1)
joblib.dump(res, f'charitable_amount_ns1.joblib')

In [ ]:
do_analysis_charitable(y = y_amount, n_splits = 5)
joblib.dump(res, f'charitable_amount_ns5.joblib')

In [ ]:
do_analysis_charitable(y = y_gave, n_splits = 1)
joblib.dump(res, f'charitable_gave_ns1.joblib')

In [ ]:
do_analysis_charitable(y = y_gave, n_splits = 5)
joblib.dump(res, f'charitable_gave_ns5.joblib')

# Synthetic Data Results
*This runs the analysis on synthetic data, corresponding to Tables 1-6 in the paper. DGP 0 is the "simple design", DGP 3 is the "nonlinear design", and DGP 1 is the "high-dimensional design".*



In [ ]:
def do_analysis_charitable(target_dir, dgp, n_samples_list, sample_its, n_jobs, gcv):
    pluginlg_experiments(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its, n_jobs=n_jobs, gcv_reg=gcv)
    pluginrf_experiments(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its, n_jobs=n_jobs, gcv_reg=gcv)
    splin_experiments(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its, n_jobs=n_jobs, gcv_reg=gcv)
    nystrom_advkernel_experiments_1000(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its, n_jobs=n_jobs, gcv_reg=gcv)
    nystrom_advkernel_experiments(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its, n_jobs=n_jobs, gcv_reg=gcv)
    rf_experiments(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its, n_jobs=n_jobs, gcv_reg=gcv)
    nnet_experiments(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its, n_jobs=n_jobs, gcv_reg=gcv)

    res = [pluginlg_postprocess(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its),
           pluginrf_postprocess(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its),
           splin_postprocess(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its),
           nystrom_advkernel_postprocess_1000(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its),
           nystrom_advkernel_postprocess(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its),
           rf_postprocess(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its),
           nnet_postprocess(n_samples_list, dgp=dgp, target_dir=target_dir, sample_its=sample_its)]

In [ ]:
do_analysis_charitable(target_dir = '', dgp = 0, n_samples_list = [100, 200, 500, 1000, 2000], sample_its = 100, n_jobs = -1, gcv = True)
joblib.dump(res, f'synthetic_dgp0.joblib')

In [ ]:
do_analysis_charitable(target_dir = '', dgp = 1, n_samples_list = [1000], sample_its = 100, n_jobs = -1, gcv = True)
joblib.dump(res, f'synthetic_dgp1.joblib')

In [ ]:
do_analysis_charitable_all(target_dir = '', dgp = 3, n_samples_list = [100], sample_its = 100, n_jobs = -1, gcv = True)
joblib.dump(res, f'synthetic_dgp3.joblib')

# 401(k) Data Results (Not Included in Paper)

### Moment Function (One Treatment)

In [ ]:
# E[E[Y|D=1, X] – E[Y|D=0, X]]
# D is the first column and X is the remaining columns.
def moment_fn(x, test_fn):
    n_obs = x.shape[0]
    if torch.is_tensor(x):
        with torch.no_grad():
            t1 = torch.cat([torch.ones((n_obs, 1)).to(device), x[:, 1:]], dim=1)
            t0 = torch.cat([torch.zeros((n_obs, 1)).to(device), x[:, 1:]], dim=1)
    else:
        t1 = np.hstack([np.ones((n_obs, 1)), x[:, 1:]])
        t0 = np.hstack([np.zeros((n_obs, 1)), x[:, 1:]])
    return test_fn(t1) - test_fn(t0)

### Adversarial Riesz Estimators (One Treatment)

In [ ]:
  def get_lg_plugin_fn(X):
      clf = LogisticRegressionCV(cv=3, max_iter=10000, random_state=123)
      C_ = clf.fit(X[:, 1:], X[:, 0]).C_[0]
      model_t_treat = LogisticRegression(C=C_, max_iter=10000, random_state=123)
      return lambda: PluginRR(model_t=model_t_treat,
                              min_propensity=1e-6)

  def get_rf_plugin_fn(X):
      gcv = GridSearchCV(RandomForestClassifier(bootstrap=True, random_state=123),
                        param_grid={'max_depth': [3, None],
                                    'min_samples_leaf': [10, 50]},
                        scoring='r2',
                        cv=5)
      best_model_treat = clone(clone(gcv).fit(X[:, 1:], X[:, 0]).best_estimator_)
      return lambda: PluginRR(model_t=best_model_treat,
                              min_propensity=1e-6)

  def get_rf_fn(X):
      return lambda: AdvEnsembleReisz(moment_fn=moment_fn,
                                      n_treatments=1,
                                      max_abs_value=15,
                                      n_iter=40,
                                      degree=1)

### Load, process, and perform estimation on datasets
*The observations of the 401(k) data are partitioned into quintile groups of household income. `q = 0` refers to the whole, unsubsetted data. In order to help satify the common support assumption, the data are trimmed in the sense that untreated observations with propensity scores (estimated according to a multinomial logistic model) outside of the hull of the propensity scores of the treated observations are dropped.*

In [ ]:
n_splits = 1
res = {}

for q in np.arange(0, 6):
    res[f'q={q}'] = {}
    print(f'Quintile={q}')
    df = pd.read_csv(f'401k/quintile{q}_trimmed.csv', index_col=0)
    y = df['Y'].values
    X = df[['D'] + [f'X{i}' for i in np.arange(1, 10)]].values

    # scale non-binary variables
    y_amount = y_amount.astype(np.double)
    y_gave = y_gave.astype(np.double)
    X = X.astype(np.double)
    idx_nonbi = [i for i in range(2, X.shape[1]) if type_of_target(X[:, i]) != 'binary']  # indices of non-binary variables (first and second columns should be binary)
    X[:, idx_nonbi] = StandardScaler().fit_transform(X[:, idx_nonbi])
    y_scale = np.std(y_amount)
    y_amount = y_amount / y_scale

    # shuffle data (doing it once at the start is more efficient than repeatedly doing it; the cross-validation here and elsewhere assumes that the data is shuffled)
    inds = np.arange(X.shape[0])
    np.random.seed(123)
    np.random.shuffle(inds)
    X, y = X[inds].copy(), y[inds].copy()

    # filter extreme propensities
    clf = LogisticRegressionCV(cv=5, max_iter=10000, random_state=123).fit(X[:, 1:], X[:, 0])
    prop = clf.predict_proba(X[:, 1:])
    filt = (prop[:, 1] <= .9) & (prop[:, 1] >= .1)
    print(X.shape[0], np.sum(filt))
    X, y = X[filt], y[filt]

    for name, get_reisz_fn in [
                                ('plugin_lg', get_lg_plugin_fn),
                                ('plugin_rf', get_rf_plugin_fn),
                                ('splin', get_splin_fn),
                                # ('advrkhs', get_advkernel_fn),
                                ('nys_advrkhs_1000', get_advnyskernel_fn_1000),
                                ('nys_advrkhs', get_advnyskernel_fn),
                                ('advrf', get_rf_fn),
                                ('advnnet', get_agmm_fn)
                                ]:
        est = DebiasedMoment(moment_fn=moment_fn,
                             get_reisz_fn=get_reisz_fn,
                             get_reg_fn=get_reg_fn, n_splits=n_splits)
        est.fit(X, y)
        p, s, l, u = est.avg_moment()
        res[f'q={q}'][name] = {'point': p * y_scale, 'stderr': s * y_scale,
                               'lower': l * y_scale, 'upper': u * y_scale}
        p, s, l, u = est.avg_moment(tmle=True)
        res[f'q={q}'][f'{name}_tmle'] = {'point': p * y_scale, 'stderr': s * y_scale,
                                         'lower': l * y_scale, 'upper': u * y_scale}

    print(res[f'q={q}'])
    res[f'q={q}'] = pd.DataFrame(res[f'q={q}']).transpose()
res = pd.concat(res)

joblib.dump(res, f'401k_ns{n_splits}.joblib')

In [ ]:
n_splits = 5
res = {}

for q in np.arange(0, 6):
    res[f'q={q}'] = {}
    print(f'Quintile={q}')
    df = pd.read_csv(f'401k/quintile{q}_trimmed.csv', index_col=0)
    y = df['Y'].values
    X = df[['D'] + [f'X{i}' for i in np.arange(1, 10)]].values

    # scale non-binary variables
    y_amount = y_amount.astype(np.double)
    y_gave = y_gave.astype(np.double)
    X = X.astype(np.double)
    idx_nonbi = [i for i in range(2, X.shape[1]) if type_of_target(X[:, i]) != 'binary']  # indices of non-binary variables (first and second columns should be binary)
    X[:, idx_nonbi] = StandardScaler().fit_transform(X[:, idx_nonbi])
    y_scale = np.std(y_amount)
    y_amount = y_amount / y_scale

    # shuffle data (doing it once at the start is more efficient than repeatedly doing it; the cross-validation here and elsewhere assumes that the data is shuffled)
    inds = np.arange(X.shape[0])
    np.random.seed(123)
    np.random.shuffle(inds)
    X, y = X[inds].copy(), y[inds].copy()

    # filter extreme propensities
    clf = LogisticRegressionCV(cv=5, max_iter=10000, random_state=123).fit(X[:, 1:], X[:, 0])
    prop = clf.predict_proba(X[:, 1:])
    filt = (prop[:, 1] <= .9) & (prop[:, 1] >= .1)
    print(X.shape[0], np.sum(filt))
    X, y = X[filt], y[filt]

    for name, get_reisz_fn in [
                                ('plugin_lg', get_lg_plugin_fn),
                                ('plugin_rf', get_rf_plugin_fn),
                                ('splin', get_splin_fn),
                                # ('advrkhs', get_advkernel_fn),
                                ('nys_advrkhs_1000', get_advnyskernel_fn_1000),
                                ('nys_advrkhs', get_advnyskernel_fn),
                                ('advrf', get_rf_fn),
                                ('advnnet', get_agmm_fn)
                                ]:
        est = DebiasedMoment(moment_fn=moment_fn,
                             get_reisz_fn=get_reisz_fn,
                             get_reg_fn=get_reg_fn, n_splits=n_splits)
        est.fit(X, y)
        p, s, l, u = est.avg_moment()
        res[f'q={q}'][name] = {'point': p * y_scale, 'stderr': s * y_scale,
                               'lower': l * y_scale, 'upper': u * y_scale}
        p, s, l, u = est.avg_moment(tmle=True)
        res[f'q={q}'][f'{name}_tmle'] = {'point': p * y_scale, 'stderr': s * y_scale,
                                         'lower': l * y_scale, 'upper': u * y_scale}

    print(res[f'q={q}'])
    res[f'q={q}'] = pd.DataFrame(res[f'q={q}']).transpose()
res = pd.concat(res)

joblib.dump(res, f'401k_ns{n_splits}.joblib')